## Tensorboard Tutorial

This tutorial will guid you on how to use the Tensorboard. Tensorboard is an amazing utility that allows us to visualize data and how it behaves. In this tutorial, you will see for what sort of purposes you can use the Tensorboard when training a neural network. 

First you will be explained how to start the Tensorboard, followed by an enlisting of the different views offered in the Tesorboard. Next you can learn how you can visualize scalar values produced during computations, on the Tensorboard. You will also see how this provide insights to the model to fix any potential errors in the learning. Thereafter you will investigate how you can visualize vectors or collections of data as histograms using the Tensorboard. With this view you will compare how weight initialization of the neural network affects the weight update of the neural network during the learning.

In [1]:
# Make sure that you have all these libaries available to run the code successfully
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json 
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from tensorflow.examples.tutorials.mnist import input_data

c:\users\thushan\documents\python_virtualenvs\tensorflow_venv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Starting Tensorboard
To visualize things via Tensorboard, you first need to start the Tensorboard service. For that,

1. Open up the command prompt (Windows) or a terminal (Ubuntu/Mac)
2. Go into the project home directory
3. If you are using python virtuanenv, activate the virtual environment you have installed TensorFlow
4.  Make sure you can see the TensorFlow library through Python. For that,
 * Type in `python3`, you will get a ">>>" looking prompt
 * Try `import tensorflow as tf`
 * If you can run this successfully you are fine
5. Exit the python prompt (that is, ">>>") by typing `exit()` and type in the following command
 * `tensorboard --logdir=summaries`
 * `--logdir` is the directory you will create data to visualize
 * Files that Tensorboard save data into are called *event files*
 * Type of data saved into the event files is called *summary data*
 * Optionally you can use `--port=<port_you_like>` to change the port Tensorboard runs on 
6. You should now get the following message
 * TensorBoard 1.6.0 at &lt;url&gt;:6006 (Press CTRL+C to quit)
7. Enter the &lt;url&gt;:6006 in to the web browser
 * You should be able to see a orange dashboard at this point. You won't have anything to display because you haven't generated data.

**Note**: Tensorboard does not like to see multiple event files in the same directory. This can lead to you getting very gruesome curves on the display. So for each different example you create a separate folder (e.g. summaries/first, summaries/second, ...) to save data. Another thing to keep in mind is that, if you want to re-run an experiment (that is, saving event file to an already populated folder), you have to make sure to first delete the existing event files.

## Different Views of Tensorboard

Different views take inputs of different formats and display them differently. You can change the view on the top orange bar on the Tensorboard
* **Scalars** - Visualize scalar values (e.g. classification accuracy)
* **Graph** - Visualize the computational graph of your model (e.g. neural network model)
* **Distributions** - Visualize how data changes over time (e.g. weights of a neural network)
* **Histograms** - A fancier view of the distribution that shows distributions in a 3-dimensional perspective
* Projector - Can be used to visualize word embeddings (that is, word embeddings are numerical representations of words that capture their semantic relationships)
* Image - Visualizing image data
* Audio - Visualizing audio data
* Text - Visualizing text (string) data

In this tutorial you will cover the views shown in bold.

## Understanding the Benefit of Scalar Visualization through Tensorboard

In this section, you will first understand why visualizing certain things (e.g. loss or accuracy) is beneficial. When training deep neural networks, one of the crucial issues that strikes the beginners is the lack of understanding the effects of various design choices and hyperparameters. For example, if you carelessly initialize weights of a deep neural network to have a very large variance between weights, your model will quickly diverge and collapse. On the other hand, things can go wrong even when you are quite competent in taming neural networks to make use of them. For example, not paying attention to the learning rate can lead to either the divergence of the model or pre-maturely saturating to sub-optimal performance. 

One way quickly detect problems with your model is to have a graphical visualization of what's going on in our model in real time (for example, every 100 iterations). So if your model is behaving oddly, it will be clearly visible. That is exactly what tensorboard provides you with. You can decide which values needs to be displayed on the Tensorboard and Tensorboard will keep maintain a real time visualization of those values during learning.

You start by first creating a five-layer neural network that you will use to classify hand-written digit images. For that you will use the famous MNIST dataset. TensorFlow provides a simple API to load MNIST data, so you don't have to manually download data. Before that you define a simple method (that is, `accuracy()`) that calculates the accuracy of some predictions with respect to the true labels.

In [2]:
def accuracy(predictions,labels):
    '''
    Accuracy of a given set of predictions of size (N x n_classes) and
    labels of size (N x n_classes)
    '''
    return np.sum(np.argmax(predictions,axis=1)==np.argmax(labels,axis=1))*100.0/labels.shape[0]

### Define Inputs, Outputs, Weights and Biases

First you define a `batch_size` denoting the amount of data you sample at a single optimization/validation or testing step. Then you define the `layer_ids`, which gives an identifier for each of the layers of the neural network you will be defining. You then can define `layer_sizes`. Note that `len(layer_sizes)` should be `len(layer_ids)+1`, because `layer_sizes` includes the size of the input at the beginning. MNIST has images of size 28x28, which will be 784 when unwrapped to a single dimension. Then you can define the input and label placeholders, that you will later use to train the model. Finally you define two TensorFlow variables for each layer (that is, `weights` and `bias`).

You can use variable scoping (more information [here](https://www.tensorflow.org/programmers_guide/variables)) so that the variables will be nicely named and will be much easier to access later.

In [3]:
batch_size = 100
layer_ids = ['hidden1','hidden2','hidden3','hidden4','hidden5','out']
layer_sizes = [784, 500, 400, 300, 200, 100, 10]

tf.reset_default_graph()

# Inputs and Labels
train_inputs = tf.placeholder(tf.float32, shape=[batch_size, layer_sizes[0]], name='train_inputs')
train_labels = tf.placeholder(tf.float32, shape=[batch_size, layer_sizes[-1]], name='train_labels')

# Weight and Bias definitions
for idx, lid in enumerate(layer_ids):
    
    with tf.variable_scope(lid):
        w = tf.get_variable('weights',shape=[layer_sizes[idx], layer_sizes[idx+1]], 
                            initializer=tf.truncated_normal_initializer(stddev=0.05))
        b = tf.get_variable('bias',shape= [layer_sizes[idx+1]], 
                            initializer=tf.random_uniform_initializer(-0.1,0.1))
        

### Calculate Logits, Predictions, Loss and Optimization

With the input/output placeholders, weights and biases of each layer defined, you now can define the calculations to calculate the logits of the neural network. Logits are the unnormalized values produced at the last layer of the neural network. When normalized, you call them predictions. This involves iterating through each layer in the neural network and computing `tf.matmul(h,w) +b`. You also need to apply an activation function as `tf.nn.relu(tf.matmul(h,w) +b)`, for all layers except for the last layer.

Next you define loss function that is used to optimize the neural network. In this example, you can use the cross entropy loss, which often deliver better results in classification problems than the mean squared error.

Finally you will need to define an optimizer that takes in the loss and update the weights of the neural network in the direction that minimizes the loss.

In [4]:
# Calculating Logits
h = train_inputs
for lid in layer_ids:
    with tf.variable_scope(lid,reuse=True):
        w, b = tf.get_variable('weights'), tf.get_variable('bias')
        if lid != 'out':
          h = tf.nn.relu(tf.matmul(h,w)+b,name=lid+'_output')
        else:
          h = tf.nn.xw_plus_b(h,w,b,name=lid+'_output')

tf_predictions = tf.nn.softmax(h, name='predictions')
# Calculating Loss
tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=train_labels, logits=h),name='loss')

# Optimizer 
tf_learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
optimizer = tf.train.MomentumOptimizer(tf_learning_rate,momentum=0.9)
grads_and_vars = optimizer.compute_gradients(tf_loss)
tf_loss_minimize = optimizer.minimize(tf_loss)


### Defining Tensorboard Summaries

Here you can define the `tf.summary` objects. `tf.summary` objects are the type of entities understood by the Tensorboard. This means that whatever value you'd like to be displayed on the Tensorboard, you should encapsulate it as a `tf.summary` object. There are several different types of summaries. Here as you are visualizing only scalars, you can define `tf.summary.scalar` objects. Furthermore, you can use `tf.name_scope` to group scalars on the Tensorboard. That is, scalars having the same name scope will be displayed on the same row on the Tensorboard. Here you define three different summaries.

* `tf_loss_summary` : You feed in a value by means of a placeholder, whenever you need to publish this to the Tensorboard
* `tf_accuracy_summary` : You feed in a value by means of a placeholder, whenever you need to publish this to the Tensorboard
* `tf_gradnorm_summary` : This calculates the l2 norm of the gradients of the last layer of your neural network. Gradient norm is a good indicator of whether the weights of the neural network are being properly updated. A too small gradient norm can indicate *vanishing gradient* or a too large gradient can imply *exploding gradient* phenomenon.

In [5]:
# Name scope allows you to group various summaries together
# Summaries having the same name_scope will be displayed on the same row on the Tensorboard
with tf.name_scope('performance'):
    # Summaries need to display on the Tensorboard
    # Whenever need to record the loss, feed the mean loss to this placeholder
    tf_loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary') 
    # Create a scalar summary object for the loss so Tensorboard knows how to display it
    tf_loss_summary = tf.summary.scalar('loss', tf_loss_ph)

    # Whenever you need to record the loss, feed the mean test accuracy to this placeholder
    tf_accuracy_ph = tf.placeholder(tf.float32,shape=None, name='accuracy_summary') 
    # Create a scalar summary object for the accuracy so Tensorboard knows how to display it
    tf_accuracy_summary = tf.summary.scalar('accuracy', tf_accuracy_ph)

# Gradient norm summary
for g,v in grads_and_vars:
    if 'hidden5' in v.name and 'weights' in v.name:
        with tf.name_scope('gradients'):
            tf_last_grad_norm = tf.sqrt(tf.reduce_mean(g**2))
            tf_gradnorm_summary = tf.summary.scalar('grad_norm', tf_last_grad_norm)
            break
# Merge all summaries together
performance_summaries = tf.summary.merge([tf_loss_summary,tf_accuracy_summary])


### Executing the neural network model: Loading Data, Training, Validation and Testing

In the code below you do the following. First you create a session, in which you execute the operations you defined above. Then you create folder for saving summary data. You next create a summary write `summ_writer`. You can now initialize all variables. This will be followed by loading the MNIST dataset.

Then for each epoch, and each batch in training data (that is, each iteration). Execute `gradnorm_summary` if it is the first iteration and write `gradnorm_summary` to event file with summary writer. You now execute model optimization and calculating the loss. After you go through the full training dataset for a single epoch, calculate average training loss.

You follow a similar treatment for the validation dataset as well. Specifically, for each batch in validation data, you calculate validation accuracy for each batch. Thereafter calculate average validation accuracy for full validation set.

Finally, the testing phase is executed. In this, for each batch in test data, you calculate test accuracy for each batch. With that, you calculate average test accuracy for full test set. At the very end you execute `performance_summaries` and write them to event file with the summary writer.

In [6]:

image_size = 28
n_channels = 1
n_classes = 10
n_train = 55000
n_valid = 5000
n_test = 10000
n_epochs = 25

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # making sure Tensorflow doesn't overflow the GPU

session = tf.InteractiveSession(config=config)

if not os.path.exists('summaries'):
    os.mkdir('summaries')
if not os.path.exists(os.path.join('summaries','first')):
    os.mkdir(os.path.join('summaries','first'))

summ_writer = tf.summary.FileWriter(os.path.join('summaries','first'), session.graph)

tf.global_variables_initializer().run()

accuracy_per_epoch = []
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)


for epoch in range(n_epochs):
    loss_per_epoch = []
    for i in range(n_train//batch_size):
        
        # =================================== Training for one step ========================================
        batch = mnist_data.train.next_batch(batch_size)    # Get one batch of training data
        if i == 0:
            # Only for the first epoch, get the summary data
            # Otherwise, it can clutter the visualization
            l,_,gn_summ = session.run([tf_loss,tf_loss_minimize,tf_gradnorm_summary],
                                      feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                                 train_labels: batch[1],
                                                tf_learning_rate: 0.0001})
            summ_writer.add_summary(gn_summ, epoch)
        else:
            # Optimize with training data
            l,_ = session.run([tf_loss,tf_loss_minimize],
                              feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                         train_labels: batch[1],
                                         tf_learning_rate: 0.0001})
        loss_per_epoch.append(l)
        
    print('Average loss in epoch %d: %.5f'%(epoch,np.mean(loss_per_epoch)))    
    avg_loss = np.mean(loss_per_epoch)
    
    # ====================== Calculate the Validation Accuracy ==========================
    valid_accuracy_per_epoch = []
    for i in range(n_valid//batch_size):
        valid_images,valid_labels = mnist_data.validation.next_batch(batch_size)
        valid_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: valid_images.reshape(batch_size,image_size*image_size)})
        valid_accuracy_per_epoch.append(accuracy(valid_batch_predictions,valid_labels))
        
    mean_v_acc = np.mean(valid_accuracy_per_epoch)
    print('\tAverage Valid Accuracy in epoch %d: %.5f'%(epoch,np.mean(valid_accuracy_per_epoch)))
    
    # ===================== Calculate the Test Accuracy ===============================
    accuracy_per_epoch = []
    for i in range(n_test//batch_size):
        test_images, test_labels = mnist_data.test.next_batch(batch_size)
        test_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: test_images.reshape(batch_size,image_size*image_size)}
        )
        accuracy_per_epoch.append(accuracy(test_batch_predictions,test_labels))
        
    print('\tAverage Test Accuracy in epoch %d: %.5f\n'%(epoch,np.mean(accuracy_per_epoch)))
    avg_test_accuracy = np.mean(accuracy_per_epoch)
    
    # Execute the summaries defined above
    summ = session.run(performance_summaries, feed_dict={tf_loss_ph:avg_loss, tf_accuracy_ph:avg_test_accuracy})

    # Write the obtained summaries to the file, so it can be displayed in the Tensorboard
    summ_writer.add_summary(summ, epoch)
    
session.close()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Average loss in epoch 0: 2.30480
	Average Valid Accuracy in epoch 0: 8.24000
	Average Test Accuracy in epoch 0: 8.67000

Average loss in epoch 1: 2.30277
	Average Valid Accuracy in epoch 1: 8.52000
	Average Test Accuracy in epoch 1: 8.87000

Average loss in epoch 2: 2.30079
	Average Valid Accuracy in epoch 2: 8.84000
	Average Test Accuracy in epoch 2: 9.21000

Average loss in epoch 3: 2.29883
	Average Valid Accuracy in epoch 3: 9.36000
	Average Test Accuracy in epoch 3: 9.64000

Average loss in epoch 4: 2.29682
	Average Valid Accuracy in epoch

###  Visualizing the Computational Graph
First you will see what the computational graph for our model looks like. You can access this view by clicking on the **Graphs** view on the Tensorboard. It should look like something below. You can see that you have a nice flow from `train_inputs` to `loss` and `predictions` flowing through the **hidden layers** **1** to **5**.

<img src="tensorboard_graph.png" style="width: 400px;"/>

### Visualizing the Summary Data: Is Everything A-Okey Here?

MNIST classification is one of the simplest examples, and still cannot solve it with a 5 layer neural network. For MNIST, it's not difficult to achieve an accuracy of more than 90% in less than 5 epochs. So what is  going on here? Let's turn towards our Tensorboard.

This is what the Tensorboard looks like for our example.
<img src="tensorboard_1.png" style="width: 400px;"/>

### Observations and Conclusions from the Tensorboard
You can see that the accuracy is going up, but very slow. You can see that the gradients updates are increasing over time. This is an odd behavior. If you're reaching towards convergence, you should see the gradients diminishing (approaching zero), not increasing.  But because the accuracy is going up, we're on the right path. *You probably need a higher learning rate*. You can now try a learning rate of **0.01**.

###  Using a Higher Learning Rate and Executing the Neural Network Model

In the code below you do the following. First you create a session, in which you execute the operations you defined above. Then you create folder for saving summary data. You next create a summary write `summ_writer`. You can now initialize all variables. This will be followed by loading the MNIST dataset.

Then for each epoch, and each batch in training data (that is, each iteration). Execute `gradnorm_summary` if it is the first iteration and write `gradnorm_summary` to event file with summary writer. You now execute model optimization and calculating the loss. After you go through the full training dataset for a single epoch, calculate average training loss.

You follow a similar treatment for the validation dataset as well. Specifically, for each batch in validation data, you calculate validation accuracy for each batch. Thereafter calculate average validation accuracy for full validation set.

Finally, the testing phase is executed. In this, for each batch in test data, you calculate test accuracy for each batch. With that, you calculate average test accuracy for full test set. At the very end you execute `performance_summaries` and write them to event file with the summary writer.

In [7]:

image_size = 28
n_channels = 1
n_classes = 10
n_train = 55000
n_valid = 5000
n_test = 10000
n_epochs = 25

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # making sure Tensorflow doesn't overflow the GPU

session = tf.InteractiveSession(config=config)

if not os.path.exists('summaries'):
    os.mkdir('summaries')
if not os.path.exists(os.path.join('summaries','second')):
    os.mkdir(os.path.join('summaries','second'))
    
summ_writer_2 = tf.summary.FileWriter(os.path.join('summaries','second'), session.graph)

tf.global_variables_initializer().run()

accuracy_per_epoch = []
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)


for epoch in range(n_epochs):
    loss_per_epoch = []
    for i in range(n_train//batch_size):
        
        # =================================== Training for one step ========================================
        batch = mnist_data.train.next_batch(batch_size)    # Get one batch of training data
        if i == 0:
            # Only for the first epoch, get the summary data
            # Otherwise, it can clutter the visualization
            l,_,gn_summ = session.run([tf_loss,tf_loss_minimize,tf_gradnorm_summary],
                                      feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                                 train_labels: batch[1],
                                                tf_learning_rate: 0.01})
            summ_writer_2.add_summary(gn_summ, epoch)
        else:
            # Optimize with training data
            l,_ = session.run([tf_loss,tf_loss_minimize],
                              feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                         train_labels: batch[1],
                                         tf_learning_rate: 0.01})
        loss_per_epoch.append(l)
        
    print('Average loss in epoch %d: %.5f'%(epoch,np.mean(loss_per_epoch)))    
    avg_loss = np.mean(loss_per_epoch)
    
    # ====================== Calculate the Validation Accuracy ==========================
    valid_accuracy_per_epoch = []
    for i in range(n_valid//batch_size):
        valid_images,valid_labels = mnist_data.validation.next_batch(batch_size)
        valid_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: valid_images.reshape(batch_size,image_size*image_size)})
        valid_accuracy_per_epoch.append(accuracy(valid_batch_predictions,valid_labels))
        
    mean_v_acc = np.mean(valid_accuracy_per_epoch)
    print('\tAverage Valid Accuracy in epoch %d: %.5f'%(epoch,np.mean(valid_accuracy_per_epoch)))
    
    # ===================== Calculate the Test Accuracy ===============================
    accuracy_per_epoch = []
    for i in range(n_test//batch_size):
        test_images, test_labels = mnist_data.test.next_batch(batch_size)
        test_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: test_images.reshape(batch_size,image_size*image_size)}
        )
        accuracy_per_epoch.append(accuracy(test_batch_predictions,test_labels))
        
    print('\tAverage Test Accuracy in epoch %d: %.5f\n'%(epoch,np.mean(accuracy_per_epoch)))
    avg_test_accuracy = np.mean(accuracy_per_epoch)
    
    # Execute the summaries defined above
    summ = session.run(performance_summaries, feed_dict={tf_loss_ph:avg_loss, tf_accuracy_ph:avg_test_accuracy})

    # Write the obtained summaries to the file, so it can be displayed in the Tensorboard
    summ_writer_2.add_summary(summ, epoch)
    
session.close()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Average loss in epoch 0: 0.96963
	Average Valid Accuracy in epoch 0: 93.56000
	Average Test Accuracy in epoch 0: 92.70000

Average loss in epoch 1: 0.18322
	Average Valid Accuracy in epoch 1: 94.68000
	Average Test Accuracy in epoch 1: 94.10000

Average loss in epoch 2: 0.11146
	Average Valid Accuracy in epoch 2: 97.10000
	Average Test Accuracy in epoch 2: 96.59000

Average loss in epoch 3: 0.07916
	Average Valid Accuracy in epoch 3: 97.02000
	Average Test Accuracy in epoch 3: 96.59000

Average loss in epoch 4: 0.05842
	Average Valid Accuracy in epoch 4: 97.74000
	Average Test Accuracy in epoch 4: 97.38000

Average loss in epoch 5: 0.04314
	Average Valid Accuracy in epoch 5: 97.72000
	Average Test Accuracy in epoch 5: 97.45000

Average loss in epoch 6: 0.03279
	Average Valid Accuracy in epoch 6:

### Second Look at the Tensorboard: Looks Much Better Now

Now you can see that the accuracy starts close to 100 and continues to go up. And you can see that the gradient updates are also diminishing over time and approaches zero. Things seems much better with the learning rate of 0.01.

<img src="tensorboard_2.png" style="width: 400px;"/>

Next let's move beyond scalars. You will see how you can analyze vectors/collections of scalars with the Tensorboard

## Beyond Scalars: Visualizing Histograms/Distributions through Tensorboard

You saw the benefit of visualizing scalars through Tensorboard, which allowed us to see how the model behaves and fix any potential issues with the model. Moreover, visualizing the graph allowed us to see that there is an uninterrupted link from the inputs to the predictions, which is necessary for gradient calcualtions. 

Now we're going to see another useful view in Tensorboard; histograms or distributions. Histograms means what it means! It is a collection of values represented by the frequency/density that the value is present in the collection. How can you can use histograms to visualize something in the neural network. You can use histograms to visualize the network weight values over time. Visualizing network weights is important, because if the weights are wildly jumping here and their during learning, it indicates something wrong with the weight initialization or the learning rate. You will see how weights change in our example. If you look at the code, it uses a *truncated_normal_initializer(...)* to initialize weights.

###  Defining Histogram Summaries to Visualize Weights and Biases

Here you again define the `tf.summary` objects. However now you are visualizing vectors of scalars so you need to define `tf.summary.histogram` objects. In this case, you define two histogram objects (namely, `tf_w_hist` and `tf_b_hist`) that contains weights and biases of agiven layer. You will define such histogram objects for all the layers and each layer will have it's own name scope. Finally you can use the `tf.summary.merge` operation to create a grouped operation that execute all these summaries at once.

In [8]:

# Summaries need to display on the Tensorboard
# Create a summary for each weight bias in each layer
all_summaries = []
for lid in layer_ids:
    with tf.name_scope(lid+'_hist'):
        with tf.variable_scope(lid,reuse=True):
            w,b = tf.get_variable('weights'), tf.get_variable('bias')

            # Create a scalar summary object for the loss so Tensorboard knows how to display it
            tf_w_hist = tf.summary.histogram('weights_hist', tf.reshape(w,[-1]))
            tf_b_hist = tf.summary.histogram('bias_hist', b)
            all_summaries.extend([tf_w_hist, tf_b_hist])

# Merge all parameter histogram summaries together
tf_param_summaries = tf.summary.merge(all_summaries)



### Executing the neural network model (with Histogram Summaries)

In the code below you do the following. First you create a session, in which you execute the operations you defined above. Then you create folder for saving summary data. You next create a summary write `summ_writer`. You can now initialize all variables. This will be followed by loading the MNIST dataset.

Then for each epoch, and each batch in training data (that is, each iteration). Execute `gradnorm_summary` and `tf_param_summaries` if it is the first iteration and write `gradnorm_summary` and `tf_param_summaries` to event file with summary writer. You now execute model optimization and calculating the loss. After you go through the full training dataset for a single epoch, calculate average training loss.

You follow a similar treatment for the validation dataset as well. Specifically, for each batch in validation data, you calculate validation accuracy for each batch. Thereafter calculate average validation accuracy for full validation set.

Finally, the testing phase is executed. In this, for each batch in test data, you calculate test accuracy for each batch. With that, you calculate average test accuracy for full test set. At the very end you execute `performance_summaries` and write them to event file with the summary writer.

**Note**: This is as same as you did before, but here you have few additional line to compute the histogram summaries (that is, `tf_param_summaries`).


In [9]:

image_size = 28
n_channels = 1
n_classes = 10
n_train = 55000
n_valid = 5000
n_test = 10000
n_epochs = 25

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # making sure Tensorflow doesn't overflow the GPU

session = tf.InteractiveSession(config=config)

if not os.path.exists('summaries'):
    os.mkdir('summaries')
if not os.path.exists(os.path.join('summaries','third')):
    os.mkdir(os.path.join('summaries','third'))
    
summ_writer_3 = tf.summary.FileWriter(os.path.join('summaries','third'), session.graph)

tf.global_variables_initializer().run()

accuracy_per_epoch = []
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)


for epoch in range(n_epochs):
    loss_per_epoch = []
    for i in range(n_train//batch_size):
        
        # =================================== Training for one step ========================================
        batch = mnist_data.train.next_batch(batch_size)    # Get one batch of training data
        if i == 0:
            # Only for the first epoch, get the summary data
            # Otherwise, it can clutter the visualization
            l,_,gn_summ, wb_summ = session.run([tf_loss,tf_loss_minimize,tf_gradnorm_summary, tf_param_summaries],
                                      feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                                 train_labels: batch[1],
                                                tf_learning_rate: 0.00001})
            summ_writer_3.add_summary(gn_summ, epoch)
            summ_writer_3.add_summary(wb_summ, epoch)
        else:
            # Optimize with training data
            l,_ = session.run([tf_loss,tf_loss_minimize],
                              feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                         train_labels: batch[1],
                                         tf_learning_rate: 0.01})
        loss_per_epoch.append(l)
        
    print('Average loss in epoch %d: %.5f'%(epoch,np.mean(loss_per_epoch)))    
    avg_loss = np.mean(loss_per_epoch)
    
    # ====================== Calculate the Validation Accuracy ==========================
    valid_accuracy_per_epoch = []
    for i in range(n_valid//batch_size):
        valid_images,valid_labels = mnist_data.validation.next_batch(batch_size)
        valid_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: valid_images.reshape(batch_size,image_size*image_size)})
        valid_accuracy_per_epoch.append(accuracy(valid_batch_predictions,valid_labels))
        
    mean_v_acc = np.mean(valid_accuracy_per_epoch)
    print('\tAverage Valid Accuracy in epoch %d: %.5f'%(epoch,np.mean(valid_accuracy_per_epoch)))
    
    # ===================== Calculate the Test Accuracy ===============================
    accuracy_per_epoch = []
    for i in range(n_test//batch_size):
        test_images, test_labels = mnist_data.test.next_batch(batch_size)
        test_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: test_images.reshape(batch_size,image_size*image_size)}
        )
        accuracy_per_epoch.append(accuracy(test_batch_predictions,test_labels))
        
    print('\tAverage Test Accuracy in epoch %d: %.5f\n'%(epoch,np.mean(accuracy_per_epoch)))
    avg_test_accuracy = np.mean(accuracy_per_epoch)
    
    # Execute the summaries defined above
    summ = session.run(performance_summaries, feed_dict={tf_loss_ph:avg_loss, tf_accuracy_ph:avg_test_accuracy})

    # Write the obtained summaries to the file, so it can be displayed in the Tensorboard
    summ_writer_3.add_summary(summ, epoch)
    
session.close()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Average loss in epoch 0: 0.94838
	Average Valid Accuracy in epoch 0: 93.76000
	Average Test Accuracy in epoch 0: 93.41000

Average loss in epoch 1: 0.17797
	Average Valid Accuracy in epoch 1: 96.00000
	Average Test Accuracy in epoch 1: 95.43000

Average loss in epoch 2: 0.11237
	Average Valid Accuracy in epoch 2: 96.84000
	Average Test Accuracy in epoch 2: 96.67000

Average loss in epoch 3: 0.07718
	Average Valid Accuracy in epoch 3: 97.36000
	Average Test Accuracy in epoch 3: 97.08000

Average loss in epoch 4: 0.05755
	Average Valid Accuracy in epoch 4: 97.64000
	Average Test Accuracy in epoch 4: 97.63000

Average loss in epoch 5: 0.04365
	Average Valid Accuracy in epoch 5: 97.78000
	Average Test Accuracy in epoch 5: 97.41000

Average loss in epoch 6: 0.03195
	Average Valid Accuracy in epoch 6:

### Visualizing Histogram Data of Weights and Biases

Here's what our weights and biases look like. First you have 3 axis; time (x axis), value (y axis) and frequency/density of values (z axis). Darker histograms represent old data and lighter historgrams represent newer data. Higher value on the z axis means that the vector contains more values near that specific value.

**Note**: You also have an "overlay" view of the histograms over time as well. You can change the type of display on the left side option panel. 

<img src="tensorboard_3_1.png" style="width: 400px;"/>
<img src="tensorboard_3_2.png" style="width: 400px;"/>
<img src="tensorboard_3_3.png" style="width: 400px;"/>

### Effect of Different Initializers: Changing the Initialization of Weights and Re-Defining the Model

Now, instead of using `truncated_normal_initializer()`, you will use the `xavier_initialzer()` to initialize weights. Xavier initialization is much better initialization technique, especially for deep neural networks. This is because Xavier initializer, instead of using a user defined standard deviation (as you did when using the `truncated_normal_initializer()`), Xaiver initialization automatically decides the standard deviation based on the number of input and output connections to a layer. This helps to flow gradients from top to bottom without issues like *vanishing gradient*. You then define the model again.

First you define a `batch_size` denoting the amount of data you sample at a single optimization/validation or testing step. Then you define the `layer_ids`, which gives an identifier for each of the layers of the neural network you will be defining. You then can define `layer_sizes`. Note that `len(layer_sizes)` should be `len(layer_ids)+1`, because `layer_sizes` includes the size of the input at the beginning. MNIST has images of size 28x28, which will be 784 when unwrapped to a single dimension. Then you can define the input and label placeholders, that you will later use to train the model. Finally you define two TensorFlow variables for each layer (that is, `weights` and `bias`).

**Note**: This is identical to the code you used first time, except for the initialization technique used for the weights

In [10]:
batch_size = 100
layer_ids = ['hidden1','hidden2','hidden3','hidden4','hidden5','out']
layer_sizes = [784, 500, 400, 300, 200, 100, 10]

tf.reset_default_graph()

# Inputs and Labels
train_inputs = tf.placeholder(tf.float32, shape=[batch_size, layer_sizes[0]], name='train_inputs')
train_labels = tf.placeholder(tf.float32, shape=[batch_size, layer_sizes[-1]], name='train_labels')

# Weight and Bias definitions
for idx, lid in enumerate(layer_ids):
    
    with tf.variable_scope(lid):
        w = tf.get_variable('weights',shape=[layer_sizes[idx], layer_sizes[idx+1]], 
                            initializer=tf.contrib.layers.xavier_initializer())
        b = tf.get_variable('bias',shape= [layer_sizes[idx+1]], 
                            initializer=tf.random_uniform_initializer(-0.1,0.1))
        

### Calculate Logits, Predictions, Loss and Optimization

With the input/output placeholders, weights and biases of each layer defined, you now can define the calculations to calculate the logits of the neural network. Logits are the unnormalized values produced at the last layer of the neural network. When normalized, you call them predictions. This involves iterating through each layer in the neural network and computing `tf.matmul(h,w) +b`. You also need to apply an activation function as `tf.nn.relu(tf.matmul(h,w) +b)`, for all layers except for the last layer.

Next you define loss function that is used to optimize the neural network. In this example, you can use the cross entropy loss, which often deliver better results in classification problems than the mean squared error.

Finally you will need to define an optimizer that takes in the loss and update the weights of the neural network in the direction that minimizes the loss.

**Note**: This is identical to the code you used first time you defined these operations and tensors.

In [11]:
# Calculating Logits
h = train_inputs
for lid in layer_ids:
    with tf.variable_scope(lid,reuse=True):
        w, b = tf.get_variable('weights'), tf.get_variable('bias')
        if lid != 'out':
          h = tf.nn.relu(tf.matmul(h,w)+b,name=lid+'_output')
        else:
          h = tf.nn.xw_plus_b(h,w,b,name=lid+'_output')

tf_predictions = tf.nn.softmax(h, name='predictions')
# Calculating Loss
tf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=train_labels, logits=h),name='loss')

# Optimizer 
tf_learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
optimizer = tf.train.MomentumOptimizer(tf_learning_rate,momentum=0.9)
grads_and_vars = optimizer.compute_gradients(tf_loss)
tf_loss_minimize = optimizer.minimize(tf_loss)


### Defining Tensorboard Summaries

Here you can define the `tf.summary` objects. `tf.summary` objects are the type of entities understood by the Tensorboard. This means that whatever value you'd like to be displayed on the Tensorboard, you should encapsulate it as a `tf.summary` object. There are several different types of summaries. Here as you are visualizing only scalars, you can define `tf.summary.scalar` objects. Furthermore, you can use `tf.name_scope` to group scalars on the Tensorboard. That is, scalrs having the same name scope will be displayed on the same row on the Tensorboard. Here you define three different summaries.

* `tf_loss_summary` : You feed in a value by means of a placeholder, whenever you need to publish this to the Tensorboard
* `tf_accuracy_summary` : You feed in a value by means of a placeholder, whenever you need to publish this to the Tensorboard
* `tf_gradnorm_summary` : This calculates the l2 norm of the gradients of the last layer of your neural network. Gradient norm is a good indicator of whether the weights of the neural network are being properly updated. A too small gradient norm can indicate *vanishing gradient* or a too large gradient can imply *exploding gradient* phenomenon.

**Note**: This is identical to the code you used first time you defined these operations and tensors.

In [12]:
# Name scope allows you to group various summaries together
# Summaries having the same name_scope will be displayed on the same row on the Tensorboard
with tf.name_scope('performance'):
    # Summaries need to display on the Tensorboard
    # Whenever you need to record the loss, feed the mean loss to this placeholder
    tf_loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary') 
    # Create a scalar summary object for the loss so Tensorboard knows how to display it
    tf_loss_summary = tf.summary.scalar('loss', tf_loss_ph)

    # Whenever you need to record the loss, feed the mean test accuracy to this placeholder
    tf_accuracy_ph = tf.placeholder(tf.float32,shape=None, name='accuracy_summary') 
    # Create a scalar summary object for the accuracy so Tensorboard knows how to display it
    tf_accuracy_summary = tf.summary.scalar('accuracy', tf_accuracy_ph)

# Gradient norm summary
for g,v in grads_and_vars:
    if 'hidden5' in v.name and 'weights' in v.name:
        with tf.name_scope('gradients'):
            tf_last_grad_norm = tf.sqrt(tf.reduce_mean(g**2))
            tf_gradnorm_summary = tf.summary.scalar('grad_norm', tf_last_grad_norm)
            break
# Merge all summaries together
performance_summaries = tf.summary.merge([tf_loss_summary,tf_accuracy_summary])

### Defining Histogram Summaries to Visualize Weights and Biases 

Here you again define the `tf.summary` objects. However now you are visualizing vectors of scalars so you need to define `tf.summary.histogram` objects. In this case, you define two histogram objects (namely, `tf_w_hist` and `tf_b_hist`) that contains weights and biases of agiven layer. You will define such histogram objects for all the layers and each layer will have it's own name scope. Finally you can use the `tf.summary.merge` operation to create a grouped operation that execute all these summaries at once.

**Note**: This is identical to the code you used first time you defined these operations and tensors.

In [13]:

# Summaries need to display on the Tensorboard
# Create a summary for each weight bias in each layer
all_summaries = []
for lid in layer_ids:
    with tf.name_scope(lid+'_hist'):
        with tf.variable_scope(lid,reuse=True):
            w,b = tf.get_variable('weights'), tf.get_variable('bias')

            # Create a scalar summary object for the loss so Tensorboard knows how to display it
            tf_w_hist = tf.summary.histogram('weights_hist', tf.reshape(w,[-1]))
            tf_b_hist = tf.summary.histogram('bias_hist', b)
            all_summaries.extend([tf_w_hist, tf_b_hist])

# Merge all parameter histogram summaries together
tf_param_summaries = tf.summary.merge(all_summaries)


### Executing the neural network model

In the code below you do the following. First you create a session, in which you execute the operations you defined above. Then you create folder for saving summary data. You next create a summary write `summ_writer`. You can now initialize all variables. This will be followed by loading the MNIST dataset.

Then for each epoch, and each batch in training data (that is, each iteration). Execute `gradnorm_summary` and `tf_param_summaries` if it is the first iteration and write `gradnorm_summary` and `tf_param_summaries` to event file with summary writer. You now execute model optimization and calculating the loss. After you go through the full training dataset for a single epoch, calculate average training loss.

You follow a similar treatment for the validation dataset as well. Specifically, for each batch in validation data, you calculate validation accuracy for each batch. Thereafter calculate average validation accuracy for full validation set.

Finally, the testing phase is executed. In this, for each batch in test data, you calculate test accuracy for each batch. With that, you calculate average test accuracy for full test set. At the very end you execute `performance_summaries` and write them to event file with the summary writer.

**Note**: This is as same as you did before.

In [14]:

image_size = 28
n_channels = 1
n_classes = 10
n_train = 55000
n_valid = 5000
n_test = 10000
n_epochs = 25

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9 # making sure Tensorflow doesn't overflow the GPU

session = tf.InteractiveSession(config=config)

if not os.path.exists('summaries'):
    os.mkdir('summaries')
if not os.path.exists(os.path.join('summaries','fourth')):
    os.mkdir(os.path.join('summaries','fourth'))
    
summ_writer_4 = tf.summary.FileWriter(os.path.join('summaries','fourth'), session.graph)

tf.global_variables_initializer().run()

accuracy_per_epoch = []
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)


for epoch in range(n_epochs):
    loss_per_epoch = []
    for i in range(n_train//batch_size):
        
        # =================================== Training for one step ========================================
        batch = mnist_data.train.next_batch(batch_size)    # Get one batch of training data
        if i == 0:
            # Only for the first epoch, get the summary data
            # Otherwise, it can clutter the visualization
            l,_,gn_summ, wb_summ = session.run([tf_loss,tf_loss_minimize,tf_gradnorm_summary, tf_param_summaries],
                                      feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                                 train_labels: batch[1],
                                                tf_learning_rate: 0.01})
            summ_writer_4.add_summary(gn_summ, epoch)
            summ_writer_4.add_summary(wb_summ, epoch)
        else:
            # Optimize with training data
            l,_ = session.run([tf_loss,tf_loss_minimize],
                              feed_dict={train_inputs: batch[0].reshape(batch_size,image_size*image_size),
                                         train_labels: batch[1],
                                         tf_learning_rate: 0.01})
        loss_per_epoch.append(l)
        
    print('Average loss in epoch %d: %.5f'%(epoch,np.mean(loss_per_epoch)))    
    avg_loss = np.mean(loss_per_epoch)
    
    # ====================== Calculate the Validation Accuracy ==========================
    valid_accuracy_per_epoch = []
    for i in range(n_valid//batch_size):
        valid_images,valid_labels = mnist_data.validation.next_batch(batch_size)
        valid_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: valid_images.reshape(batch_size,image_size*image_size)})
        valid_accuracy_per_epoch.append(accuracy(valid_batch_predictions,valid_labels))
        
    mean_v_acc = np.mean(valid_accuracy_per_epoch)
    print('\tAverage Valid Accuracy in epoch %d: %.5f'%(epoch,np.mean(valid_accuracy_per_epoch)))
    
    # ===================== Calculate the Test Accuracy ===============================
    accuracy_per_epoch = []
    for i in range(n_test//batch_size):
        test_images, test_labels = mnist_data.test.next_batch(batch_size)
        test_batch_predictions = session.run(
            tf_predictions,feed_dict={train_inputs: test_images.reshape(batch_size,image_size*image_size)}
        )
        accuracy_per_epoch.append(accuracy(test_batch_predictions,test_labels))
        
    print('\tAverage Test Accuracy in epoch %d: %.5f\n'%(epoch,np.mean(accuracy_per_epoch)))
    avg_test_accuracy = np.mean(accuracy_per_epoch)
    
    # Execute the summaries defined above
    summ = session.run(performance_summaries, feed_dict={tf_loss_ph:avg_loss, tf_accuracy_ph:avg_test_accuracy})

    # Write the obtained summaries to the file, so it can be displayed in the Tensorboard
    summ_writer_4.add_summary(summ, epoch)
    
session.close()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Average loss in epoch 0: 0.44695
	Average Valid Accuracy in epoch 0: 95.92000
	Average Test Accuracy in epoch 0: 95.50000

Average loss in epoch 1: 0.13685
	Average Valid Accuracy in epoch 1: 96.78000
	Average Test Accuracy in epoch 1: 96.28000

Average loss in epoch 2: 0.08945
	Average Valid Accuracy in epoch 2: 97.18000
	Average Test Accuracy in epoch 2: 97.14000

Average loss in epoch 3: 0.06410
	Average Valid Accuracy in epoch 3: 97.54000
	Average Test Accuracy in epoch 3: 97.56000

Average loss in epoch 4: 0.04689
	Average Valid Accuracy in epoch 4: 98.06000
	Average Test Accuracy in epoch 4: 97.75000

Average loss in epoch 5: 0.03310
	Average Valid Accuracy in epoch 5: 97.98000
	Average Test Accuracy in epoch 5: 97.83000

Average loss in epoch 6: 0.02627
	Average Valid Accuracy in epoch 6:

### Comparing Different Initialization Techniques

Here you compare how weights evolve over time for the two different initalizations; *truncated_normal_initializer* (red) and *xavier_initializer* (blue). You can see that xavier initializer keeps more weights away from zero than the normal initializer, which is a better thing to do. This is potentially allowing the xavier initialized neural networks to converge faster, as evident by the loss/accuracy curves. 

<img src="tensorboard_3_and_4.jpg" style="height: 600px; float:left;"/>


## Distibution View of Histograms

You now can compare the difference between the two views; histogram view and the distribution view. Distribution view is essentiall a different way of looking at the histograms. If you look at the image below, you can easily see that the distribution view is a top view of the histogram view (I have rotated the histogram graphs to easily see the resemblance).

<img src="tensorboard_histogram_vs_distribution_views.png" style="height: 400px; float:left;"/>


## Conclusion

In this tutorial you saw how to use the Tensorboard. First you learnt how to start the Tensorboard through the command prompt (Windows) or terminal (Ubuntu/Mac). Next you looked at different views of data provided by the Tensorboard. You then looked at code that visualizes scalar values (e.g. loss / accuracy). You used a feed-forward neural network model to concretely understand the use of the scalar value visualization. Thereafter, you explored how you can visualize collections/vectors of scalars using the histogram view. This was followed by a comparison highlighting the differences between neural network weight initialization techniques using the histogram view. Finally you discussed the similarities between the distribution view and the histogram view.

* Author: Thushan Ganegedara
* Email: thushv@gmail.com
* Website: http://www.thushv.com/
* LinkedIn: https://www.linkedin.com/in/thushanganegedara/